<a href="https://colab.research.google.com/github/Indravaraj/resume_screening/blob/main/Copy_of_AIcruiter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required dependencies

*   **PyPDF2**: Reads and extracts text from PDF documents.
*   **pydata-google-auth**: Simplifies logging in to Google.
*  **groq**: Helps us connect with AI models to analyze and process text.
*  **langchain**: Makes it easy to create tasks using AI by building smart text-based workflows.















In [1]:
!pip install PyPDF2 pydata-google-auth groq langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 9.4 MB/s eta 0:00:00


# Imports and Their Functions

**Authentication and Google API:**
*  pydata_google_auth, build, HttpError: These are used to handle Google login, manage authentication tokens, and connect with Google services like Gmail and Calendar.

**AI and Prompt Management:**
*  Groq, PromptTemplate: These are used to interact with AI models and create structured prompts (questions) for the AI to understand and process.

**File Handling:**
*   PdfReader, files: These help read text from PDF files and allow file uploads in Google Colab.

**Utilities:**
*   datetime, base64, os: These are used to work with dates and times, encode data (for example, in emails), and manage files and directories.





In [2]:
from groq import Groq
from langchain.prompts import PromptTemplate
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from PyPDF2 import PdfReader
import datetime
import base64
import os
import pydata_google_auth
from google.colab import files

# Define Global Variables
*   **api_key**: API key for the Groq AI service.
*   **SCOPES**: Permissions for accessing Google Calendar and Gmail APIs.



In [6]:
api_key = "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"

In [7]:
SCOPES = ['https://www.googleapis.com/auth/calendar', 'https://www.googleapis.com/auth/gmail.send']

# Google Authentication:

*  **Purpose**: Authenticate access to Google services using OAuth2.
*  **Key Steps**:
  *   Fetch user credentials for specified SCOPES.
  *   Return credentials for use in Gmail and Calendar API operations.









In [5]:
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os
import json

def authenticate_google():
    creds = None

    # Check if token.json exists (previously saved credentials)
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
        print("Token File Exists!!")

    # If there are no (valid) credentials, request new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Use InstalledAppFlow for authentication
            flow = InstalledAppFlow.from_client_secrets_file(
                "/content/client_secret_586382823389-ni09423rct7s8kgdjiv6fvqvgkbm7k01.apps.googleusercontent.com.json", SCOPES
            )
            # Generate the authorization URL
            flow.redirect_uri = "https://poditivity.com"
            auth_url, _ = flow.authorization_url(prompt="consent")
            print(f"Please visit this URL to authorize this application: {auth_url}")

            # Prompt the user to enter the authorization code from the redirected URL
            code = input("Enter the authorization code: ")

            # Fetch the token using the authorization code
            flow.fetch_token(code=code)
            creds = flow.credentials

        # Save the credentials for the next run
        with open("token.json", "w") as token_file:
            json.dump({
                "token": creds.token,
                "refresh_token": creds.refresh_token,
                "token_uri": creds.token_uri,
                "client_id": creds.client_id,
                "client_secret": creds.client_secret,
                "scopes": creds.scopes
            }, token_file)

    return creds

In [8]:
authenticate_google()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=586382823389-ni09423rct7s8kgdjiv6fvqvgkbm7k01.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fpoditivity.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=S1HOO4BzSuJT4DnsVkYh2UHz96LQvX&prompt=consent&access_type=offline
Enter the authorization code: 4/0AeanS0ZWr5z8GFTyKgfR498jj8JC0XMhq-hsT4kJujxg1YnoD7WgwbtGXN-niixzNfCFKA


# Extract text from PDF

*   **Purpose**: Extracts text content from a PDF.
*   **Key Steps**:
  *   Reads the PDF file.
  *   Concatenates text from all pages into a single string.





In [9]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = "".join(page.extract_text() for page in reader.pages)
    return text

# Assess Suitability

*   **Purpose**: Uses AI to determine if a candidate matches the job description.
*   **Key Steps**:
  *   Creates a structured prompt for the AI model.
  *   Sends the query to the Groq API.
  *   Returns a simple "YES" or "NO" decision.







In [10]:
def assess_suitability(job_description, resume):
    # Define a prompt template
    prompt_template = PromptTemplate(
        input_variables=["resume", "job_description"],
        template=(
            "Consider yourself as an experienced talent acquisition specialist. Given the job description and the candidate's resume below, "
            "determine if the candidate is suitable for the role.\n\n"
            "Job Description:\n{job_description}\n\n"
            "Candidate Resume:\n{resume}\n\n"
            "Respond STRICTLY with 'YES' or 'NO'. 'YES' if the candidate is suitable, otherwise 'NO'."
        ),
    )

    query = prompt_template.format(job_description=job_description, resume=resume)
    client = Groq(api_key=api_key)

    response = client.chat.completions.create(
        messages=[{
                "role": "system",
                "content": "You are a talent acquisition specialist, analyze the resume against the job description provided."
            },
            {
                "role": "user",
                "content": query
            }
        ],
        model="llama3-8b-8192",
    )
    print(response)

    return response.choices[0].message.content

# Send an email to specific email ID with given subject and body
*   **Purpose**: Sends an email using Gmail API.
*   **Key Steps**:
  *   Encodes the email message.
  *   Sends it through the Gmail service.

In [11]:
def send_email(service, to_email, subject, body):
    message = {
        "raw": base64.urlsafe_b64encode(
            f"To: {to_email}\nSubject: {subject}\n\n{body}".encode("utf-8")
        ).decode("utf-8")
    }
    try:
        service.users().messages().send(userId="me", body=message).execute()
    except HttpError as error:
        print(f"An error occurred: {error}")

# Schedule an invitation for the Interview if the candidate is shortlisted
*   **Purpose**: Creates a calendar event and generates a Google Meet link.
*   **Key Steps**:
  *   Schedules a 30-minute interview for the next day.
  *   Sends event details to Google Calendar.

In [12]:
def schedule_interview(service, candidate_email, candidate_name):
    now = datetime.datetime.utcnow()
    start_time = now + datetime.timedelta(days=1, hours=3)  # Set for next day
    end_time = start_time + datetime.timedelta(minutes=30)

    event = {
        "summary": f'Interview with {candidate_name}',
        "location": "Google Meet",
        "description": "Discussion regarding the job application.",
        "start": {"dateTime": start_time.isoformat() + "Z", "timeZone": "UTC"},
        "end": {"dateTime": end_time.isoformat() + "Z", "timeZone": "UTC"},
        "conferenceData": {
            "createRequest": {
                "requestId": "sample123",
                "conferenceSolutionKey": {"type": "hangoutsMeet"},
            }
        },
        "attendees": [{"email": candidate_email}],
    }
    try:
        event_result = service.events().insert(
            calendarId="primary",
            body=event,
            conferenceDataVersion=1,
        ).execute()

        return event_result.get("hangoutLink")
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

# Resume Screening
*   **Purpose**: Coordinates the full process.
*   **Key Steps**:
  *   Authenticates Google APIs.
  *   Extracts text from PDFs.
  *   Uses AI to evaluate the candidate.
  *   Sends emails based on the decision.
  *   If selected, schedules an interview.

In [13]:
def resume_screening(job_description_pdf, resume_pdf, candidate_email, candidate_name):
    creds = authenticate_google()
    gmail_service = build("gmail", "v1", credentials=creds)
    calendar_service = build("calendar", "v3", credentials=creds)

    job_description = extract_text_from_pdf(job_description_pdf)
    resume = extract_text_from_pdf(resume_pdf)

    decision = assess_suitability(job_description, resume)

    print(decision)

    if decision == "NO":
        send_email(
            gmail_service,
            to_email=candidate_email,
            subject="Application Update",
            body=f"Dear, {candidate_name}\n\n"
            "Thank you for applying. Unfortunately, your skills do not match the role requirements at this time. We wish you the best of luck in your career!",
        )
    elif decision == "YES":
        meet_link = schedule_interview(calendar_service, candidate_email, candidate_name)
        if meet_link:
            send_email(
                gmail_service,
                to_email=candidate_email,
                subject="Interview Invitation",
                body=(
                    f"Congratulations {candidate_name}!\n\n"
                    "You've been shortlisted for the role.\n"
                    f"Please join us for an interview at the following link: {meet_link}"
                ),
            )

# User Interaction

*   **Purpose**: Accepts user inputs and starts the screening process.
*   **Key Steps**:
  *   Prompts for job description and resume uploads.
  *   Collects candidate details.
  *   Initiates the resume_screening function.

**Uploading Job Description**

In [14]:
print("Upload job description PDF")
job_desc_file = files.upload()
job_desc_path = next(iter(job_desc_file.keys()))

Upload job description PDF


Saving Frontend Engineer - Intern.pdf to Frontend Engineer - Intern.pdf



**Uploading candidate resume & entering details**

In [15]:
print("Upload candidate resume PDF")
resume_file = files.upload()
resume_path = next(iter(resume_file.keys()))
candidate_email = input("Enter candidate's email: ")
candidate_name=input("Enter candidate's name: ")


Upload candidate resume PDF


Saving front-end-developer-resume-example.pdf to front-end-developer-resume-example.pdf
Enter candidate's email: indravaraj345@gmail.com
Enter candidate's name: indravaraj


**Final Step**

In [16]:
resume_screening(job_desc_path, resume_path, candidate_email,candidate_name)

Token File Exists!!
ChatCompletion(id='chatcmpl-52c4aa42-351b-4552-860a-74d8976d7e3e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='YES', role='assistant', function_call=None, tool_calls=None))], created=1733594500, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_6a6771ae9c', usage=CompletionUsage(completion_tokens=2, prompt_tokens=1405, total_tokens=1407, completion_time=0.001666667, prompt_time=0.182294324, queue_time=-0.0013703130000000119, total_time=0.183960991), x_groq={'id': 'req_01jeh4hk16fbdr7eg7edpgtf6q'})
YES
